In [1]:
import datetime

import pandas as pd
import numpy as np
import glob

import src.DataGrabber
from src.utils import *


In [83]:
trials_with_cv = pd.read_csv("fix_cv_07_16_23_v2.csv")
# df = pd.read_csv("reasonable_models_to_test_testing_07_10_23_v4.csv")
# df
trials_with_cv.columns = ["index", "Kernel", "BIC", "Error", "Error_Data", "Hyperparameters", "data", "model_dictionary"]
trials_with_cv

,index,Kernel,BIC,Error,Error_Data,Hyperparameters,data,model_dictionary
0,0,RQ+AR2+Mat_2.5,-2393.656006,0.078362,"[0.012824076227843761, 0.03202127665281296, 0....","[['kernels.0.raw_outputscale', 0.0001192597264...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
1,1,RQ+AR2+Mat_2.5+Per_Season*Mat_1.5,-2346.249756,0.079316,"[0.01150863990187645, 0.04021153599023819, 0.0...","[['kernels.0.raw_outputscale', 0.0024466121103...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
2,2,RQ+AR2+Mat_2.5+Per_Season*Mat_0.5,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[['kernels.0.raw_outputscale', nan], ['kernels...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
3,3,RQ+AR2+Mat_2.5+Per_Season*Mat_2.5,-2257.091553,0.080293,"[0.02663501165807247, 0.01363297551870346, 0.0...","[['kernels.0.raw_outputscale', 0.0012184567749...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
4,4,RQ+AR2+Mat_2.5+Per_Year,-2395.350342,0.077351,"[0.009948301129043102, 0.04533999785780907, 0....","[['kernels.0.raw_outputscale', 0.0027659246698...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
5,5,RQ+AR2+Mat_2.5+Per_Season,-2291.658936,0.079996,"[0.01231719832867384, 0.016856368631124496, 0....","[['kernels.0.raw_outputscale', 0.0005249131354...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
6,6,RQ+AR2+Mat_2.5+Per_Month,-2361.827148,0.077915,"[0.014917830936610699, 0.03195611760020256, 0....","[['kernels.0.raw_outputscale', 0.0004875638696...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
7,7,RQ+AR2+Mat_2.5+Per_Week,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[['kernels.0.raw_outputscale', nan], ['kernels...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
8,8,RQ+AR2+Mat_2.5+Per_Season*Mat_1.5+Per_Year*RBF,-2233.409912,0.078630,"[0.020471978932619095, 0.016076043248176575, 0...","[['kernels.0.raw_outputscale', 0.0025181076489...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...
9,9,RQ+AR2+Mat_2.5+Per_Season*Mat_1.5+Per_Month*RBF,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[['kernels.0.raw_outputscale', nan], ['kernels...","{'train_x': array([7.9366320e-04, 9.2750118e-0...",OrderedDict([('likelihood.noise_covar.raw_nois...


In [153]:
import math
shorter_table = trials_with_cv.loc[:, ["Kernel", "BIC", "Error"]].sort_values(by="BIC", inplace=False)
# print(shorter_table.to_latex(
#     index=False, formatters={"Kernel": lambda x: x.replace("_", "").replace("'", "").replace(" ", "")},
#     float_format="{:0.4f}".format,
# ))
# print(trials_with_cv.loc[0, ["Kernel", "Hyperparameters", "model_dictionary"]][1].replace(", ", "\n"))
list_of_hypers = trials_with_cv.loc[:, ["Kernel", "BIC", "Hyperparameters", "model_dictionary"]].sort_values(by="BIC", inplace=False)\
    .iloc[0,2]\
    .replace("'", "").replace("]'", "").replace("'","")\
    .replace("[", "").replace("raw_", "").replace("]", "")\
    .split(", ")
actual_hypers = {
    "outputscale":[None]*len(list_of_hypers),
    "alpha":[None]*len(list_of_hypers),
    "lengthscale":[None]*len(list_of_hypers),
    "period":[None]*len(list_of_hypers),
    "period_length":[None]*len(list_of_hypers), }
for i in range(len(list_of_hypers)-1):
    current = list_of_hypers[i]
    next = list_of_hypers[i+1]
    if "kernel" in current:
        kernel_index = int(current.split(".")[1])
        hyper_type = current.split(".")[-1]
        if hyper_type in current:
            if hyper_type == "outputscale":
                actual_hypers[hyper_type][kernel_index] = math.exp(float(next))

            elif hyper_type == "lengthscale":
                in_days = (float(next)*1e9/ 60 / 60 / 24)
                if in_days > 600:
                    actual_hypers[hyper_type][kernel_index] = in_days / 365#) + " years"
                else:
                    actual_hypers[hyper_type][kernel_index] = in_days#) + " days"
            elif hyper_type == "period":
                actual_hypers[hyper_type][kernel_index] = float(next)*1e9 / 60 / 60 / 24#)+ " days"
            elif  hyper_type == "period_length":
                actual_hypers[hyper_type][kernel_index] = float(next)#/ 60 / 60 / 24 / 365)+ " years"
            else:
                actual_hypers[hyper_type][kernel_index] = float(next)
        else:
            actual_hypers["other"].append([kernel_index, float(next)])
        # else:
        #     actual_hypers.append(float(next))
        # print(current, " : ", float(next))
# print(shorter_table)
hyper_table = pd.DataFrame(actual_hypers).T
h_table = hyper_table.iloc[:, :4]
h_table.columns = ["RQ", "AR2", "Mat2.5", "PerYear"]
print(h_table.to_latex(
    index=True,
    formatters={},
    float_format="{:0.2f}".format,))
h_table
# print(h_table.to_latex(
#     index=True, formatters={},
#     float_format="{:0.4f}".format,
# ))
# trials_with_cv
# trials_with_cv = pd.read_csv("top_10_trials_with_cv.csv")
# trials_with_cv = trials_with_cv.iloc[:, 1:]

# trials_with_cv.sort_values(by="BIC", inplace=True)
# trials_with_cv.columns = ["index", "BIC_rank", "Kernel", "BIC", "Error"]
# trials_with_cv.sort_values(by="Error", inplace=True)
# trials_with_cv.reset_index(drop=True, inplace=True)
# trials_with_cv["Error_rank"] = trials_with_cv.index
# trials_with_cv = trials_with_cv.loc[:, ["Kernel", "BIC_rank", "BIC", "Error", "Error_rank"]]
# trials_with_cv

\begin{tabular}{lrrrr}
\toprule
{} &    RQ &  AR2 &  Mat2.5 &  PerYear \\
\midrule
outputscale   &  1.00 & 1.02 &    1.06 &     1.01 \\
alpha         &  1.08 &  NaN &     NaN &      NaN \\
lengthscale   & 81.02 & 7.05 &    6.91 &    30.52 \\
period        &   NaN & 6.72 &     NaN &      NaN \\
period\_length &   NaN &  NaN &     NaN &     0.10 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_16663/868482995.py:50: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(h_table.to_latex(


,RQ,AR2,Mat2.5,PerYear
outputscale,1.002770,1.019313,1.061031,1.014076
alpha,1.076540,NaN,NaN,NaN
lengthscale,81.023972,7.051050,6.909257,30.524254
period,NaN,6.716245,NaN,NaN
period_length,NaN,NaN,NaN,0.096177


In [33]:
print(trials_with_cv.loc[:,["Kernel", "BIC", "Error", "BIC_rank", "Error_rank"]].style.to_latex())
print(trials_with_cv.to_latex(
    index=False,
    formatters={},
    float_format="{:0.2f}".format,))

\begin{tabular}{llrrrr}
 & Kernel & BIC & Error & BIC_rank & Error_rank \\
0 & 'AR2+RQ*AR2' & -2326.191406 & 0.095826 & 9 & 0 \\
1 & 'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF' & -2326.375732 & 0.095949 & 8 & 1 \\
2 & 'RBF+AR2+Mat_2.5' & -2333.447754 & 0.095949 & 6 & 2 \\
3 & 'RBF+AR2*Per_Year*RBF*Mat_1.5+Per_Month' & -2335.052002 & 0.096061 & 5 & 3 \\
4 & 'RBF+AR2*Per_Year*RBF*Mat_1.5*RBF' & -2350.953613 & 0.096677 & 1 & 4 \\
5 & 'RBF+AR2*Per_Year*RBF' & -2346.211914 & 0.096680 & 2 & 5 \\
6 & 'RBF+AR2*Per_Year*RBF*Mat_1.5*Mat_2.5' & -2335.255371 & 0.096698 & 4 & 6 \\
7 & 'RBF+AR2*Per_Year*RBF*Mat_1.5' & -2355.538086 & 0.096761 & 0 & 7 \\
8 & 'RBF+AR2*Per_Year' & -2337.717529 & 0.096776 & 3 & 8 \\
9 & 'RBF+AR2*Per_Year*RBF+Per_Month' & -2333.310303 & 0.096863 & 7 & 9 \\
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
                                  Kernel &  BIC\_rank &      BIC &  Error &  Error\_rank \\
\midrule
                            'AR2+RQ*AR2' &         9 & -2326.19 &   0.10 &      

/tmp/ipykernel_78900/472129381.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(trials_with_cv.to_latex(


In [34]:
def add_past_trials():
    list_of_past_trials = glob.glob('./../Past_Trials/csv_results/'+'*.csv')
    all_trials = pd.DataFrame()
    for trial in list_of_past_trials:
        print(trial)
        trial_df = pd.read_csv(trial)
        all_trials = pd.concat([all_trials,trial_df],axis=0)
    cleaned_all_trials = all_trials.loc[:, ["path_location", "Kernel", "BIC"]]
    cleaned_all_trials.to_csv("./../Past_Trials/cleaned_all_trials_2.csv", index=False)
    return cleaned_all_trials

In [38]:
cleaned_all_trials_2 = add_past_trials()
cleaned_all_trials_2.drop_duplicates(inplace=True)
cleaned_all_trials_2

./../Past_Trials/csv_results/full_path_history_2023-06-09-025823_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-05-29-19:01:58_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-05-30-15:22:27_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-06-07-015322_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-06-08-031146_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-05-30-23:04:30_cleaned.csv
./../Past_Trials/csv_results/Starting_with_RBF.csv


,path_location,Kernel,BIC
0,"(0, 0)",'RQ',-589.327698
1,"(1, 0)",'RQ+AR2+RBF',-1818.655396
2,"(2, 0)",'RQ+AR2+Mat_2.5+RBF',NaN
3,"(3, 0)",'RQ+AR2+Mat_2.5+Per_Season+RBF',-2335.488037
4,"(0, 1)",'RQ+Mat_2.5',-2026.503540
...,...,...,...
110,"(0, 22)",'RBF*Per_Month',1889.415039
111,"(1, 22)",'RBF+AR2*Per_Month',290.225006
112,"(2, 22)",'RBF+AR2*Per_Year*Per_Month',-2197.544922
113,"(3, 22)",'RBF+AR2*Per_Year*RBF*Per_Month',-1335.378174


In [4]:
# all_past_trials = pd.read_csv("./../Past_Trials/full_results/cleaned_all_trials.csv")
# all_past_trials.sort_values(by="BIC", inplace=True)
# list_of_kernels_ranked = all_past_trials.loc[:, ["Kernel"]]
# all_past_trials


In [50]:
# print(
new_kernel_potential_list = cleaned_all_trials_2 \
    .loc[cleaned_all_trials_2["BIC"]<10].sort_values(by="BIC", inplace=False).loc[:,["Kernel", "BIC"]]
        # .to_latex(
        #     index=False,
        #     formatters={"Kernel": lambda x: x.replace("_", "").replace("'", "").replace(" ", "")},
        #     float_format="{:0.2f}".format,))
new_kernel_potential_list.to_csv("new_kernel_potential_list.csv", index=False)
new_kernel_potential_list

,Kernel,BIC
38,'RQ+AR2+Mat_2.5+Per_Season',-2385.041992
5,'RQ+AR2+Mat_2.5',-2372.063965
54,'RQ+AR2+Mat_2.5*Mat_2.5',-2364.126953
73,'RBF+AR2*Per_Year*RBF*Mat_1.5',-2355.538086
39,'RQ+AR2+Mat_2.5+Per_Season+Per_Season',-2352.207520
...,...,...
67,'Min+AR2*Per_Month',-455.552368
36,'RQ+Per_Season',-345.876099
66,'AR2*Per_Month',-340.112885
89,'RQ+AR2*Per_Month',-311.452271


In [51]:
trials_with_cv = pd.read_csv("top_10_trials_with_cv.csv")
# trials_with_cv.sort_values(by="BIC", inplace=True)
trials_with_cv.columns = ["BIC_rank", "Kernel", "BIC", "Error"]
trials_with_cv.sort_values(by="Error", inplace=True)
trials_with_cv.reset_index(drop=True, inplace=True)
# trials_with_cv.columns = ["Error_rank", "BIC_rank", "Kernel", "BIC", "Error"]
trials_with_cv["Error_rank"] = trials_with_cv.index
trials_with_cv = trials_with_cv.loc[:, ["Kernel", "BIC_rank", "BIC", "Error", "Error_rank"]]
trials_with_cv

,Kernel,BIC_rank,BIC,Error,Error_rank
0,'AR2+RQ*AR2',9,-2326.191406,0.095826,0
1,'RBF+AR2*Per_Year*RBF*Mat_1.5+RBF',8,-2326.375732,0.095949,1
2,'RBF+AR2+Mat_2.5',6,-2333.447754,0.095949,2
3,'RBF+AR2*Per_Year*RBF*Mat_1.5+Per_Month',5,-2335.052002,0.096061,3
4,'RBF+AR2*Per_Year*RBF*Mat_1.5*RBF',1,-2350.953613,0.096677,4
5,'RBF+AR2*Per_Year*RBF',2,-2346.211914,0.096680,5
6,'RBF+AR2*Per_Year*RBF*Mat_1.5*Mat_2.5',4,-2335.255371,0.096698,6
7,'RBF+AR2*Per_Year*RBF*Mat_1.5',0,-2355.538086,0.096761,7
8,'RBF+AR2*Per_Year',3,-2337.717529,0.096776,8
9,'RBF+AR2*Per_Year*RBF+Per_Month',7,-2333.310303,0.096863,9


In [6]:
# trials_with_cv.sort_values(by="BIC", inplace=True)
# trials_with_cv

In [7]:
# pd.read_csv("./../Past_Trials/full_results/cleaned_all_trials.csv")

In [8]:
# dg = src.DataGrabber.DataGrab()
# dg.grab_data_block_reduce()